In [3]:
import os
import warnings
import re

import math
import sympy
import scipy
import cobra
import csv
import numpy as np
import pandas as pd
from micom import Community
from cobra.io import read_sbml_model
from cobra.medium import minimal_medium
from cobra.flux_analysis import (single_gene_deletion, single_reaction_deletion, 
                                 double_gene_deletion, double_reaction_deletion)

In [4]:
org_list_file = open('/Users/ashnanarain/Computational_Biology/Gut_Microbiome/Lethals/Single_Species.csv')
org_names = [(i.split())[0] for i in org_list_file.readlines()]

In [8]:
for org in org_names:
    model = read_sbml_model('/Users/ashnanarain/Computational_Biology/Gut_Microbiome/Models/' + org + '.xml')
    print(len(model.metabolites))

2022
1876
1744
1987
2129
2041
2204
946
926
1808
1710
2630
1600
1242
1225
1953
2154
2157
1691
2591


In [3]:
pairwise_orgs = []
for i in range(len(org_names)):
    for j in range(i + 1, len(org_names)):
        pairwise_orgs.append(org_names[i] + ',' + org_names[j])

In [4]:
lethals_pd = pd.DataFrame(columns = pairwise_orgs, index = range(1000))

In [ ]:
file = open('/Users/ashnanarain/Computational_Biology/Gut_Microbiome/FINAL_Growths.csv', 'w')
csv_writer = csv.writer(file)

In [5]:
medium_western_file = open('/Users/ashnanarain/Computational_Biology/Gut_Microbiome/Diets_and_Media/Diet_Western.csv', 'r')
medium_western = {}
for i in medium_western_file.readlines():
    
    met = i.strip().split(',')
    met[0] = met[0].replace('_e', '_m')
    medium_western[met[0]] = float(met[1]) * float(met[2])
medium_western    

{'EX_fru_m': 0.014898600000000001,
 'EX_glc_m': 0.014898600000000001,
 'EX_gal_m': 0.014898600000000001,
 'EX_man_m': 0.014898600000000001,
 'EX_mnl_m': 0.014898600000000001,
 'EX_fuc_L_m': 0.014898600000000001,
 'EX_glcn_m': 0.014898600000000001,
 'EX_rmn_m': 0.014898600000000001,
 'EX_arab_L_m': 0.0178783,
 'EX_drib_m': 0.0178783,
 'EX_rib_D_m': 0.0178783,
 'EX_xyl_D_m': 0.0178783,
 'EX_oxa_m': 0.044695700000000005,
 'EX_lcts_m': 0.007449300000000001,
 'EX_malt_m': 0.007449300000000001,
 'EX_sucr_m': 0.007449300000000001,
 'EX_melib_m': 0.007449300000000001,
 'EX_cellb_m': 0.074493,
 'EX_tre_m': 0.074493,
 'EX_strch1_m': 0.257339,
 'EX_amylopect900_m': 1.57e-05,
 'EX_amylose300_m': 4.7e-05,
 'EX_arabinan101_m': 0.000166277,
 'EX_arabinogal_m': 2.19e-05,
 'EX_arabinoxyl_m': 0.000306649,
 'EX_bglc_m': 7.05e-08,
 'EX_cellul_m': 2.82e-05,
 'EX_dextran40_m': 0.000176323,
 'EX_galmannan_m': 1.41e-05,
 'EX_glcmannan_m': 3.29e-05,
 'EX_homogal_m': 0.000128235,
 'EX_inulin_m': 0.000470195,
 '

In [10]:
rows = []
for pair in pairwise_orgs:
    print(pair)
    orgs = pair.split(',')
    
    file_1 = '/Users/ashnanarain/Computational_Biology/Gut_Microbiome/Models/' + orgs[0] + '.xml'
    file_2 = '/Users/ashnanarain/Computational_Biology/Gut_Microbiome/Models/' + orgs[1] + '.xml'  
    
    model_1 = read_sbml_model(file_1)
    model_2 = read_sbml_model(file_2)
    
    table = {'id': ['model_1', 'model_2'], 'file':[file_1, file_2]}

    data = pd.DataFrame(table)      
    taxonomy = data 
    com = Community(taxonomy, progress = False)

    com.medium = minimal_medium(com, 1)
    
    '''
    medium_western_matched = {}
    for i in medium_western.keys():
        if i not in dir(com.exchanges):
            pass
        else:
            medium_western_matched[i] = medium_western[i]
            
    com.medium = medium_western_matched
    '''
    
    sol = com.cooperative_tradeoff(fraction = 0.7, fluxes=False)
    sol_df = sol.members
    print(sol)
    

    #rows.append([orgs[0], orgs[1], len(com.reactions), len(com.metabolites), sol.growth_rate, sol_df._get_value('model_1', 'growth_rate'), sol_df._get_value('model_2', 'growth_rate'), sol_df._get_value('medium', 'reactions')])
    print([orgs[0], orgs[1], len(com.reactions), len(com.metabolites), sol.growth_rate, sol_df._get_value('model_1', 'growth_rate'), sol_df._get_value('model_2', 'growth_rate'), sol_df._get_value('medium', 'reactions')])

    '''
    non_exc_rxns = []
    for i in model.reactions:
        if i not in model.exchanges:
            non_exc_rxns.append(i)
    '''
            
    single_reaction_deletion_results = single_reaction_deletion(com)

    single_lethal_reactions = []

    for i in range(len(single_reaction_deletion_results)):
        if single_reaction_deletion_results.loc[i]['growth'] < (0.1 * sol.growth_rate):
            single_lethal_reactions.append(single_reaction_deletion_results.loc[i]['ids'])
    print(single_lethal_reactions)
    break
    lethals_pd[pair] = pd.Series(single_lethal_reactions)


Alistipes_shahii_WAL_8301,Bacteroides_caccae_ATCC_43185
[<Reaction 1HIBUPGLU_St2pp__model_1 at 0x17e6f1250>, <Reaction 1HIBUPGLU_Stex__model_1 at 0x17e6f00d0>, <Reaction 1HIBUP_S_GLCAASE__model_1 at 0x17e6f1f10>, <Reaction 1HIBUP_S_GLCAASEepp__model_1 at 0x17f72d110>, <Reaction 1HIBUP_Stepp__model_1 at 0x17e6bf390>, <Reaction 1HIBUP_Stex__model_1 at 0x17e6f3750>, <Reaction 1HMDGLUC_GLCAASE__model_1 at 0x17e6bd490>, <Reaction 1HMDGLUC_GLCAASEepp__model_1 at 0x17e6beb10>, <Reaction 1HMDGLUCt2pp__model_1 at 0x17e6bd1d0>, <Reaction 1HMDGLUCtex__model_1 at 0x17e6bf750>, <Reaction 1OHMDZtepp__model_1 at 0x17e6bcdd0>, <Reaction 1OHMDZtex__model_1 at 0x17e6bf010>, <Reaction 1P4H2CBXLAH__model_1 at 0x17e6bfcd0>, <Reaction 23DHMPO__model_1 at 0x17e6be110>, <Reaction 26DAP_Mtex__model_1 at 0x17e4c9a50>, <Reaction 2AHBUTI__model_1 at 0x17e4c97d0>, <Reaction 2DHPL__model_1 at 0x17e4c8c50>, <Reaction 2DMMQ8abcpp__model_1 at 0x17e61ee50>, <Reaction 2DMMQ8tex__model_1 at 0x17e61e210>, <Reaction 2HATVA

[01:37:54] WARNING  Could not identify an external compartment by name and choosing one with   ]8;id=227679;file:///Users/ashnanarain/anaconda3/envs/cobraenv/lib/python3.11/site-packages/cobra/medium/boundary_types.py\boundary_types.py]8;;\:]8;id=469789;file:///Users/ashnanarain/anaconda3/envs/cobraenv/lib/python3.11/site-packages/cobra/medium/boundary_types.py#90\90]8;;\
                    the most boundary reactions. That might be complete nonsense or change                         
                    suddenly. Consider renaming your compartments using `Model.compartments`                       
                    to fix this.                                                                                   

           WARNING  Could not identify an external compartment by name and choosing one with   ]8;id=622749;file:///Users/ashnanarain/anaconda3/envs/cobraenv/lib/python3.11/site-packages/cobra/medium/boundary_types.py\boundary_types.py]8;;\:]8;id=822564;file:///Users/ashnanarain/anaconda3/envs/cobraenv/lib/python3.11/site-packages/cobra/medium/boundary_types.py#90\90]8;;\
                    the most boundary reactions. That might be complete nonsense or change                         
                    suddenly. Consider renaming your compartments using `Model.compartments`                       
                    to fix this.                                                                                   

<CommunitySolution 0.700 at 0x2843bc090>
['Alistipes_shahii_WAL_8301', 'Bacteroides_caccae_ATCC_43185', 5136, 4396, 0.7000000000137384, 0.0, 1.4000000000274768, 498]


/Users/ashnanarain/anaconda3/envs/cobraenv/lib/python3.11/site-packages/cobra/medium/minimal_medium.py:112: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  medium = pd.Series()


Read LP format model from file /var/folders/dn/p3xbs4r94jqf_5q7_q53zj_c0000gn/T/tmplhyoauvt.lp
Reading time = 0.02 seconds
: 4399 rows, 10273 columns, 33327 nonzeros
Read LP format model from file /var/folders/dn/p3xbs4r94jqf_5q7_q53zj_c0000gn/T/tmp9e_wnjbe.lp
Reading time = 0.02 seconds
: 4399 rows, 10273 columns, 33327 nonzeros
Read LP format model from file /var/folders/dn/p3xbs4r94jqf_5q7_q53zj_c0000gn/T/tmp4bhyvfvr.lp
Reading time = 0.02 seconds
: 4399 rows, 10273 columns, 33327 nonzeros
Read LP format model from file /var/folders/dn/p3xbs4r94jqf_5q7_q53zj_c0000gn/T/tmppxnxs97t.lp
Reading time = 0.02 seconds
: 4399 rows, 10273 columns, 33327 nonzeros
Read LP format model from file /var/folders/dn/p3xbs4r94jqf_5q7_q53zj_c0000gn/T/tmpom36fl2p.lp
Reading time = 0.02 seconds
: 4399 rows, 10273 columns, 33327 nonzeros
Read LP format model from file /var/folders/dn/p3xbs4r94jqf_5q7_q53zj_c0000gn/T/tmp0bwp55d1.lp
Reading time = 0.02 seconds
: 4399 rows, 10273 columns, 33327 nonzeros
Read

In [13]:
for i in range(len(single_reaction_deletion_results)):
    if single_reaction_deletion_results.loc[i]['growth'] < (0.1 * sol.growth_rate):
        single_lethal_reactions.append(single_reaction_deletion_results.loc[i]['ids'])
print(single_lethal_reactions)

[{'EX_glyc3p(e)__model_1'}, {'PPCDC__model_2'}, {'PTPAT__model_2'}, {'3OAS60__model_2'}, {'EX_cspg_b(e)__model_2'}, {'CBPS__model_2'}, {'CHORS__model_2'}, {'PYDXNtex__model_2'}, {'pbiosynthesis__model_2'}, {'DAPE__model_2'}, {'IGPDH__model_2'}, {'UGMDDS__model_2'}, {'EX_fe2(e)__model_2'}, {'HISTP__model_2'}, {'GALUi__model_2'}, {'EX_mn2_m'}, {'CA2abcpp__model_2'}, {'ACPS1__model_2'}, {'EX_cspg_ab_rest_m'}, {'EX_ca2_m'}, {'ALAR__model_2'}, {'PEPGLY__model_2'}, {'EX_fe2_m'}, {'3OAR80__model_2'}, {'UMPK__model_2'}, {'FE2tex__model_2'}, {'AGPR__model_2'}, {'GALT1__model_2'}, {'EAR100x__model_2'}, {'UAPGR__model_2'}, {'U23GAAT__model_2'}, {'EX_cobalt2(e)__model_2'}, {'EX_pydxn(e)__model_2'}, {'3OAR140__model_2'}, {'GLCTR2__model_2'}, {'Cuabcpp__model_2'}, {'ATPPRT__model_2'}, {'UAGAAT__model_2'}, {'G1PTT__model_2'}, {'KDOCT2__model_2'}, {'GNCORE2tex__model_2'}, {'biomass536__model_2'}, {'TDPGDH__model_2'}, {'ACLS_a__model_2'}, {'ASPKi__model_2'}, {'EX_gncore2_m'}, {'EX_k_m'}, {'EAR60x__mode

In [ ]:
file = open('FINAL_Community_Lethals_Western_01.csv', 'w')
lethals_pd.to_csv(file, index=False)

In [ ]:
file = open('FINAL_Community_Growths_Western.csv', 'w')
growths_df = pd.DataFrame(rows)
print(growths_df)
growths_df.to_csv(file, index=False)